In [1]:
from cache_analysis.new_cache.fixponit import *

import sys
print(sys.version_info)

sys.version_info(major=3, minor=9, micro=12, releaselevel='final', serial=0)


In [2]:
f = r"D:\workspace\Gitdocuments\hw-memory\cache_analysis\new_cache\input\cache_information.in"
config, graph, user_kwargs = read_from_file(f)

print("==== Cache Config ====")
print("Set index length:", config.set_index_len)
print("Cache offset:", config.offset_len)
print("Cache association:", config.assoc)

print("\n==== TCFG ====")
for node in graph.all_nodes:
    print(node.ident,
          "access", [b.__str__() for b in node.access_blocks],
          "incoming", [n.ident for n in node.incoming],
          "outgoing", [n.ident for n in node.outgoing])

==== Cache Config ====
Set index length: 8
Cache offset: 6
Cache association: 4

==== TCFG ====
n0 access ['<MemBlk tag:0x0 idx:0x1d>', '<MemBlk tag:0x0 idx:0x1e>', '<MemBlk tag:0x0 idx:0x1e>', '<MemBlk tag:0x0 idx:0x1e>', '<MemBlk tag:0x0 idx:0x1e>', '<MemBlk tag:0x0 idx:0x1e>', '<MemBlk tag:0x4 idx:0x3f>', '<MemBlk tag:0x0 idx:0x1e>', '<MemBlk tag:0x0 idx:0x1e>', '<MemBlk tag:0x4 idx:0x3f>', '<MemBlk tag:0x0 idx:0x1e>', '<MemBlk tag:0x0 idx:0x1e>', '<MemBlk tag:0x4 idx:0x3f>', '<MemBlk tag:0x0 idx:0x1e>', '<MemBlk tag:0x4 idx:0x40>', '<MemBlk tag:0x0 idx:0x1e>', '<MemBlk tag:0x0 idx:0x1e>', '<MemBlk tag:0x4 idx:0x3f>', '<MemBlk tag:0x0 idx:0x1e>'] incoming [] outgoing ['n3']
n1 access ['<MemBlk tag:0x0 idx:0x1e>', '<MemBlk tag:0x0 idx:0x1e>', '<MemBlk tag:0x4 idx:0x3f>', '<MemBlk tag:0x0 idx:0x1e>', '<MemBlk tag:0x0 idx:0x1f>', '<MemBlk tag:0x5 idx:0xbd>', '<MemBlk tag:0x0 idx:0x1f>', '<MemBlk tag:0x0 idx:0x1f>', '<MemBlk tag:0x0 idx:0x1f>'] incoming ['n13'] outgoing ['n14']
n2 acces

In [3]:
graph_must, graph_may, graph_pers = deepcopy(graph), deepcopy(graph), deepcopy(graph)

fixpoint(config, graph_must, 'must', **user_kwargs)
fixpoint(config, graph_may, 'may', **user_kwargs)
fixpoint(config, graph_pers, 'persistent', **user_kwargs)

True

In [4]:
considered_nodes = {'n8', 'n6', 'n15', 'n7'}

mb_must_hit = list()
mb_persistent = list()
mb_must_miss = list()
mb_uc = list()

for n_must, n_may, n_pers in zip(graph_must.all_nodes, graph_may.all_nodes, graph_pers.all_nodes):
    if n_must.ident not in considered_nodes:
        continue
    for rlt_must, rlt_may, rlt_pers in zip(n_must.analysis_result(), n_may.analysis_result(), n_pers.analysis_result()):
        print(rlt_must[0])
        for mb, must_f, may_f, pers_f in zip(rlt_must[1], rlt_must[2], rlt_may[2], rlt_pers[2]):
            print(mb, must_f, may_f, pers_f)
            if must_f:
                mb_must_hit.append(mb)
            else:
                if pers_f:
                    mb_persistent.append(mb)
                else:
                    if not may_f:
                        mb_must_miss.append(mb)
                    else:
                        mb_uc.append(mb)


(2048, 2052)
<MemBlk tag:0x0 idx:0x20> True True True
(2052, 2056)
<MemBlk tag:0x0 idx:0x20> True True True
(2056, 2060)
<MemBlk tag:0x0 idx:0x20> True True True
(2060, 2064)
<MemBlk tag:0x0 idx:0x20> True True True
(2064, 2068)
<MemBlk tag:0x0 idx:0x20> True True True
(2068, 2072)
<MemBlk tag:0x0 idx:0x20> True True True
(2072, 2076)
<MemBlk tag:0x0 idx:0x20> True True True
(2076, 2080)
<MemBlk tag:0x0 idx:0x20> True True True
(2080, 2084)
<MemBlk tag:0x0 idx:0x20> True True True
(2084, 2088)
<MemBlk tag:0x0 idx:0x20> True True True
(2088, 2092)
<MemBlk tag:0x0 idx:0x20> True True True
(2220, 2224)
<MemBlk tag:0x0 idx:0x22> False True True
(2224, 2228)
<MemBlk tag:0x0 idx:0x22> True True True
(2228, 2232)
<MemBlk tag:0x0 idx:0x22> True True True
(2232, 2236)
<MemBlk tag:0x0 idx:0x22> True True True
(2236, 2240)
<MemBlk tag:0x0 idx:0x22> True True True
(2240, 2244)
<MemBlk tag:0x0 idx:0x23> False True True
(2244, 2248)
<MemBlk tag:0x0 idx:0x23> True True True
(2248, 2252)
<MemBlk tag:0

In [7]:
print("mb_must_hit: ", mb_must_hit)
print("mb_persistent: ", mb_persistent)
print("mb_must_miss: ", mb_must_miss)
print("mb_uc: ", mb_uc)

mb_must_hit:  [<MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x22>, <MemBlk tag:0x0 idx:0x22>, <MemBlk tag:0x0 idx:0x22>, <MemBlk tag:0x0 idx:0x22>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x24>, <MemBlk tag:0x0 idx:0x24>, <MemBlk tag:0x0 idx:0x24>, <MemBlk tag:0x0 idx:0x24>, <MemBlk tag:0x0 idx:0x24>, <MemBlk tag:0x0 idx:0x24>, <MemBlk tag:0

In [8]:
mb_must_hit = list()
mb_persistent = list()
mb_must_miss = list()
mb_uc = list()
print("mb_must_hit: ", mb_must_hit)
print("mb_persistent: ", mb_persistent)
print("mb_must_miss: ", mb_must_miss)
print("mb_uc: ", mb_uc)

mb_must_hit:  []
mb_persistent:  []
mb_must_miss:  []
mb_uc:  []


In [22]:
considered_nodes = {'n8', 'n6', 'n15', 'n7'}

mb_must_hit = list()
mb_persistent = list()
mb_must_miss = list()
mb_uc = list()
page_range = ((0, 4096), (65536, 69632))
for tup in page_range:
    for n_must, n_may, n_pers in zip(graph_must.all_nodes, graph_may.all_nodes, graph_pers.all_nodes):
        if n_must.ident not in considered_nodes:
            continue
        for rlt_must, rlt_may, rlt_pers in zip(n_must.analysis_result(), n_may.analysis_result(), n_pers.analysis_result()):
            # print(rlt_must[0])
            for mb, must_f, may_f, pers_f in zip(rlt_must[1], rlt_must[2], rlt_may[2], rlt_pers[2]):
            # print(mb, must_f, may_f, pers_f)
                if tup[0] <= config.block2address(mb) <= tup[1]:
                    if must_f:
                        mb_must_hit.append(mb)
                    else:
                        if pers_f:
                            mb_persistent.append(mb)
                        else:
                            if not may_f:
                                mb_must_miss.append(mb)
                            else:
                                mb_uc.append(mb)

    print(tup, "mb_must_hit: ", mb_must_hit)
    print(tup, "mb_persistent: ", mb_persistent)
    print(tup, "mb_must_miss: ", mb_must_miss)
    print(tup, "mb_uc: ", mb_uc)
    mb_must_hit = list()
    mb_persistent = list()
    mb_must_miss = list()
    mb_uc = list()


(0, 4096) mb_must_hit:  [<MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x20>, <MemBlk tag:0x0 idx:0x22>, <MemBlk tag:0x0 idx:0x22>, <MemBlk tag:0x0 idx:0x22>, <MemBlk tag:0x0 idx:0x22>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x23>, <MemBlk tag:0x0 idx:0x24>, <MemBlk tag:0x0 idx:0x24>, <MemBlk tag:0x0 idx:0x24>, <MemBlk tag:0x0 idx:0x24>, <MemBlk tag:0x0 idx:0x24>, <MemBlk tag:0x0 idx:0x24>]
(0,